# Batch anomaly detection with the Anomaly Detector API

### Use this Jupyter notebook to start visualizing anomalies as a batch with the Anomaly Detector API in Python.

This notebook shows you how to send a batch anomaly detection request, and vizualize the anomalies found throughout the example data set. The graph created at the end of this notebook will display the following:
* Anomalies found throughout the data set, highlighted.
* The expected values versus the values contained in the data set.
* Anomaly detection boundaries 


# Azure Stream Analytics Query

If you are exporting telemetry from IoT Hub/Central to ASA then this query will be a useful starting point

```sql
WITH Telemetry AS (
    SELECT
        deviceId as DeviceId,
        enrichments.deviceName as DeviceName,
        telemetry.latitude as Latitude,
        telemetry.longitude as Longitude,
        MAX(telemetry.temperature) AS Temperature,
        AVG(telemetry.humidity) AS Humidity,
        AVG(telemetry.pressure) AS Pressure,
        System.Timestamp() as Timestamp,
        Count(*) as Count
    FROM [weather-eh] TIMESTAMP BY enqueuedTime
    GROUP BY
        deviceId,
        enrichments.deviceName,
        telemetry.latitude,
        telemetry.longitude,
        TumblingWindow(minute,4)
)

SELECT DeviceId, DeviceName, LEFT(CAST(Timestamp as nvarchar(max)), 16) as timestamp, Temperature AS value INTO [anomaly-data] FROM Telemetry
```

Install the following Python libraries.

- pandas
- numpy
- bokeh
- ipywidgets
- azure.storage.blob
- matplotlib

In [1]:
# To start sending requests to the Anomaly Detector API, paste your Anomaly Detector resource access key below,
# and replace the endpoint variable with the endpoint for your region or your on-premise container endpoint. 
# Endpoint examples:
# https://westus2.api.cognitive.microsoft.com/anomalydetector/v1.0/timeseries/entire/detect
# http://127.0.0.1:5000/anomalydetector/v1.0/timeseries/entire/detect

# Output from Azure Stream Analytics
# apikey = '[REPLACE_WITH_AD_SUBSCRIPTION_KEY]'
# endpoint = '[REPLACE_WITH_AD_ENDPOINT_URL]'
# blob_conn_str="[REPLACE_WITH_BLOB_CONNECTION_STRING]"

# IoT Central Blog Storage Logging
apikey =   "d9938de38b29435f82fdba35aaba6634"
endpoint = "https://anomaly-predictive-maintenance.cognitiveservices.azure.com/anomalydetector/v1.0/timeseries/entire/detect"



In [2]:
import requests
import json
import pandas as pd
import numpy as np
import warnings
import math
# from azure.storage.blob import ContainerClient

warnings.filterwarnings('ignore')

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 

In [3]:
from bokeh.plotting import figure,output_notebook, show
from bokeh.palettes import Blues4
from bokeh.models import ColumnDataSource,Slider
import datetime
from bokeh.io import push_notebook
from dateutil import parser
from ipywidgets import interact, widgets, fixed
from IPython.display import clear_output 

output_notebook()

Loading BokehJS ...

In [4]:
def detect(endpoint, apikey, request_data):
    headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': apikey}
    response = requests.post(endpoint, data=json.dumps(request_data), headers=headers)
    if response.status_code == 200:
        return json.loads(response.content.decode("utf-8"))
    else:
        print(response.status_code)
        raise Exception(response.text)

format_json_data iterates through the blob data and adds to a python dictionary.
A dictionary is used as it dedups any duplicate timestamps. The dictionary is later converted to a 
list of json objects.

In [5]:
def build_figure(sample_data, sensitivity):
    sample_data['sensitivity'] = sensitivity
    
    result = detect(endpoint, apikey, sample_data)
        
    columns = {'expectedValues': result['expectedValues'], 'isAnomaly': result['isAnomaly'], 'isNegativeAnomaly': result['isNegativeAnomaly'],
          'isPositiveAnomaly': result['isPositiveAnomaly'], 'upperMargins': result['upperMargins'], 'lowerMargins': result['lowerMargins'],
          'timestamp': [parser.parse(x['timestamp']) for x in sample_data['series']], 
          'value': [x['value'] for x in sample_data['series']]}
    response = pd.DataFrame(data=columns)
    values = response['value']
    label = response['timestamp']
    anomalies = []
    anomaly_labels = []
    index = 0
    anomaly_indexes = []
    p = figure(x_axis_type='datetime', title="Batch Anomaly Detection ({0} Sensitvity)".format(sensitivity), width=800, height=600)
    for anom in response['isAnomaly']:
        if anom == True and (values[index] > response.iloc[index]['expectedValues'] + response.iloc[index]['upperMargins'] or 
                         values[index] < response.iloc[index]['expectedValues'] - response.iloc[index]['lowerMargins']):
            anomalies.append(values[index])
            anomaly_labels.append(label[index])
            anomaly_indexes.append(index)
        index = index+1
    upperband = response['expectedValues'] + response['upperMargins']
    lowerband = response['expectedValues'] -response['lowerMargins']
    band_x = np.append(label, label[::-1])
    band_y = np.append(lowerband, upperband[::-1])
    boundary = p.patch(band_x, band_y, color=Blues4[2], fill_alpha=0.5, line_width=1, legend_label='Boundary')
    p.line(label, values, legend_label='Value', color="#2222aa", line_width=1)
    p.line(label, response['expectedValues'], legend_label='ExpectedValue',  line_width=1, line_dash="dotdash", line_color='olivedrab')
    anom_source = ColumnDataSource(dict(x=anomaly_labels, y=anomalies))
    anoms = p.circle('x', 'y', size=5, color='tomato', source=anom_source)
    p.legend.border_line_width = 1
    p.legend.background_fill_alpha  = 0.1
    show(p, notebook_handle=True)

## Vizualizing anomalies throughout your data

The following cells call the Anomaly Detector API with two different example time series data sets, and different sensitivities for anomaly detection. Varying the sensitivity of the Anomaly Detector API can improve how well the response fits your data.

### Example 1: time series with an hourly sampling frequency


In [6]:
# Load from backup if there is a network/service problem :)

with open('iot_central_data_backup.json', 'r') as f:
    ts_dict = json.load(f)
    print(json.dumps(ts_dict, indent=4, sort_keys=True))


[
    {
        "timestamp": "2022-07-29 05:00:00+00:00",
        "value": 21
    },
    {
        "timestamp": "2022-07-29 05:05:00+00:00",
        "value": 21
    },
    {
        "timestamp": "2022-07-29 05:10:00+00:00",
        "value": 21
    },
    {
        "timestamp": "2022-07-29 05:15:00+00:00",
        "value": 21
    },
    {
        "timestamp": "2022-07-29 05:20:00+00:00",
        "value": 21
    },
    {
        "timestamp": "2022-07-29 05:25:00+00:00",
        "value": 21
    },
    {
        "timestamp": "2022-07-29 05:30:00+00:00",
        "value": 37
    },
    {
        "timestamp": "2022-07-29 05:35:00+00:00",
        "value": 37
    },
    {
        "timestamp": "2022-07-29 05:40:00+00:00",
        "value": 22
    },
    {
        "timestamp": "2022-07-29 05:45:00+00:00",
        "value": 22
    },
    {
        "timestamp": "2022-07-29 05:50:00+00:00",
        "value": 42
    },
    {
        "timestamp": "2022-07-29 05:55:00+00:00",
        "value": 43
    },
  

In [7]:
# print(ts_dict)

if len(ts_dict) < 12:
    print("Climate data items: {count}".format(count=len(ts_dict)))
    print("12 Climate data items required")
    print("Wait for more data to be generated then try again")
else:

    # Create data for anomaly detection

    sample_data = {}
    sample_data['series'] = ts_dict
    sample_data['granularity'] = 'minutely'
    sample_data['customInterval'] = 5

    # sample_data['period'] = 24
    # 95 sensitivity
    build_figure(sample_data,95)

In [8]:
# 90 sensitivity
build_figure(sample_data,90)

In [9]:
#85 sensitivity
build_figure(sample_data,85)